# Create Zone Level From Lots

Converted from `create_zone_level_from_lots.py`

In [ ]:
"""Aggregate lot-level enforcement back to zone-level for occupancy predictions"""import pandas as pdimport numpy as npprint("Loading lot-level enforcement data...")lot_level = pd.read_csv('../data/processed/enforcement_lot_level_extended.csv',                        parse_dates=['datetime'],                        dtype={'Lot_number': int})print(f"Loaded {len(lot_level):,} lot-level records")print(f"Unique lots: {lot_level['Lot_number'].nunique()}")# Aggregate by Zone_Name + datetimeprint("\nAggregating to zone level...")zone_level = lot_level.groupby(['Zone_Name', 'datetime']).agg({    'lpr_scans': 'sum',    'amp_sessions': 'sum',    'tickets_issued': 'sum',    'unpaid_estimate': 'sum',    'lpr_estimated': 'any',  # True if any lot has estimated LPR    'hour': 'first',    'year': 'first',    'month': 'first',    'day_of_week': 'first',    'is_weekend': 'first',    'time_of_day': 'first',    'time_of_day_code': 'first',    'temperature_f': 'first',    'precipitation_inches': 'first',    'snowfall_inches': 'first',    'snow_depth_inches': 'first',    'wind_mph': 'first',    'weather_code': 'first',    'weather_category': 'first',    'is_rainy': 'first',    'is_snowy': 'first',    'is_cold': 'first',    'is_hot': 'first',    'is_windy': 'first',    'is_severe': 'first',    'is_game_day': 'first',    'is_dead_week': 'first',    'is_finals_week': 'first',    'is_spring_break': 'first',    'is_thanksgiving_break': 'first',    'is_winter_break': 'first',    'is_any_break': 'first'}).reset_index()# Recalculate enforcement rate at zone levelzone_level['enforcement_rate'] = 0.0mask = zone_level['unpaid_estimate'] > 0zone_level.loc[mask, 'enforcement_rate'] = (    zone_level.loc[mask, 'tickets_issued'] / zone_level.loc[mask, 'unpaid_estimate'])# Add date columnzone_level['date'] = zone_level['datetime'].dt.date# Rename Zone_Name to Zone for consistencyzone_level = zone_level.rename(columns={'Zone_Name': 'Zone'})print(f"\nZone-level aggregation complete:")print(f"  Total records: {len(zone_level):,}")print(f"  Unique zones: {zone_level['Zone'].nunique()}")print(f"  Date range: {zone_level['datetime'].min()} to {zone_level['datetime'].max()}")# Saveoutput_file = '../data/processed/enforcement_zone_level_extended.csv'zone_level.to_csv(output_file, index=False)print(f"\nSaved: {output_file}")# Show sampleprint(f"\nSample zones:")for zone in sorted(zone_level['Zone'].unique())[:10]:    zone_data = zone_level[zone_level['Zone'] == zone]    tickets = zone_data['tickets_issued'].sum()    hours_with_enf = (zone_data['tickets_issued'] > 0).sum()    print(f"  {zone:30s}: {tickets:6,} tickets, {hours_with_enf:6,} hours with enforcement")